Laiba Zubair (24472) \\
Saad Lakhani (24471) \\
Talal Khan (25253)

In [ ]:
!pip install faiss-cpu langchain langchain-community sentence-transformers rank_bm25 pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from rank_bm25 import BM25Okapi
import numpy as np

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline

import textwrap

Chunking

In [ ]:
# Load and split PDF
doc_path = "https://homel.vsb.cz/~fai0013/Kniha_Algoritmy.pdf"
loader = PyPDFLoader(doc_path)
pages = loader.load()
print(len(pages))

# Chunking text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

# Prepare documents and their metadata
texts = [chunk.page_content for chunk in chunks]
metadata = [chunk.metadata for chunk in chunks]
print(len(texts))
print(chunks[10])

593
2905
page_content='Epilogue 471
APPENDIX A
Useful Formulas for the Analysis of Algorithms 475
APPENDIX B
Short Tutorial on Recurrence Relations 479
References 493
Hints to Exercises 503
Index 547
v' metadata={'producer': 'Acrobat Distiller 7.0.5 for Macintosh', 'creator': 'Acrobat: pictwpstops filter 1.0', 'creationdate': '2011-08-17T15:36:00+08:00', 'title': 'Introduction to the Design and Analysis of Algorithms (2-downloads)', 'author': 'Anany Levitin', 'moddate': '2011-09-21T01:33:09+02:00', 'ebx_publisher': '/Pearson Education', 'pxcviewerinfo': "PDF-XChange Viewer;2.5.197.0;Jul  8 2011;20:50:13;D:20110921013309+02'00'", 'universal pdf': 'The process that creates this PDF constitutes a trade secret of codeMantra, LLC and is protected by the copyright laws of the United States', 'codemantra, llc': 'http://www.codemantra.com', 'source': 'https://homel.vsb.cz/~fai0013/Kniha_Algoritmy.pdf', 'total_pages': 593, 'page': 6, 'page_label': 'v'}


In [ ]:
# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")

# Create FAISS vector database
vectordb = FAISS.from_documents(pages, embedding_model)

# Save FAISS index to disk for later use
vectordb.save_local("faiss_index")

# Check the number of stored documents
print(f"Number of documents in the vector store: {vectordb.index.ntotal}")

<ipython-input-4-cb5d2d317eec>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of documents in the vector store: 593


In [ ]:
# BM25 Indexing
tokenized_texts = [text.split() for text in texts]
bm25 = BM25Okapi(tokenized_texts)

def reciprocal_rank_fusion(results_bm25, results_embedding, k=2):
    scores = {}

    # Use document content or metadata as the key
    for rank, (doc, score) in enumerate(results_bm25):
        doc_id = doc.page_content  # Or use doc.metadata.get("source", "unknown") if available
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (rank+1) # (k + rank + 1)
        print("BM25", scores[doc_id])

    for rank, (doc, score) in enumerate(results_embedding):
        doc_id = doc.page_content  # Use the same identifier
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (rank+1) # (k + rank + 1)
        print("Dense", scores[doc_id])

    return sorted(scores.items(), key=lambda x: x[1], reverse=True)


# Extract page content and metadata properly
def format_response(doc):
    return f"Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content.strip()}"

In [ ]:
# Retrieve function
def retrieve(query, k=3):
    query_embedding = embedding_model.embed_query(query)
    results_embedding = vectordb.similarity_search_with_score_by_vector(query_embedding, k=k)
    results_embedding = sorted(results_embedding, key=lambda x: x[1], reverse=True)

    print("============Dense Embeddings=============")
    for doc, score in results_embedding:
        print(f"page {doc.metadata.get('page','Unknown')} - Score: {score:.4f} - {doc.page_content[:100]}...")

    # Get BM25 scores for all documents and sort to get top-k results
    results_bm25 = [(idx, bm25.get_scores(query.split())[idx]) for idx in range(len(texts))]
    results_bm25 = sorted(results_bm25, key=lambda x: x[1], reverse=True)[:k]  # Keep only top-k results
    # Convert BM25 results to (Document, score) format
    results_bm25_docs = [(Document(page_content=texts[idx], metadata=metadata[idx]), score) for idx, score in results_bm25]

    print("************BM25 Results*************")
    for doc, score in results_bm25_docs:
        print(f"page {doc.metadata.get('page','Unknown')} - Score: {score:.4f} - {doc.page_content[:100]}...")

    # Create a lookup dictionary {document content -> Document object}
    doc_lookup = {doc.page_content: doc for doc, _ in results_bm25_docs}
    doc_lookup.update({doc.page_content: doc for doc, _ in results_embedding})

    # Fuse results
    fused_results = reciprocal_rank_fusion(results_bm25_docs, results_embedding)

    # Format results, ensuring document IDs are mapped back to actual Documents
    return [format_response(doc_lookup[doc_id]) for doc_id, _ in fused_results if doc_id in doc_lookup]

    #fused_results = reciprocal_rank_fusion(results_bm25, results_embedding)
    #return [(texts[idx], metadata[idx]["page"] if "page" in metadata[idx] else "Unknown") for idx, _ in fused_results]


In [ ]:
def rag_pipeline(model_name, question, ground_truth):
  model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", #device_map='cuda'
    torch_dtype="auto",
    trust_remote_code=True,
  )
  tokenizer = AutoTokenizer.from_pretrained(model_name)


  print("Model Information")
  print(model.dtype)
  total_params = sum(p.numel() for p in model.parameters())
  print(f"Total Parameters: {total_params / 1e6} million")
  memory_footprint = total_params * 2 / (1024 ** 2)  # Convert to MB
  print(f"Estimated Memory Footprint: {memory_footprint:.2f} MB\n")

  # Create a pipeline
  generator = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
  return_full_text=False,
  max_new_tokens=5000,
  # do_sample=False
  )

  retrieved_responses = retrieve(question, k=3)
  semantic_response = ""
  for i in range(0,len(retrieved_responses)):
    semantic_response = semantic_response + (retrieved_responses[i])
    semantic_response = semantic_response + "-------"

  print("Now performing LLM Search")

  # Construct the RAG prompt
  prompt = f"""
  You are an AI assistant tasked with answering questions based on retrieved knowledge.

  ### **Retrieved Information**:
  1. {retrieved_responses[0]}

  2. {retrieved_responses[1]}

  3. {retrieved_responses[2]}

  ### **Question**:
  {question}

  ### **Instructions**:
  - Integrate the key points from all retrieved responses into a **cohesive, well-structured answer**.
  - If the responses are **contradictory**, mention the different perspectives.
  - If none of the retrieved responses contain relevant information, reply:
  **"I couldn't find a good response to your query in the database."**
  """

  # Generate response using LLM
  messages = [{"role": "user", "content": prompt}]
  output = generator(messages)
  llm_response =  textwrap.fill(output[0]["generated_text"], width=80)
  print("Responses with semantic search:\n{}\nResponses with LLM use:\n{}".format(semantic_response, llm_response))

  #EVALATUATION
  print("\n\nEVALUATION\n")
    # 🔍 Retrieval Evaluation Prompt
  eval_prompt = f"""
  You are an expert evaluator.

  ### Task:
  Assess the quality of the retrieved information used to answer the question.

  ### Question:
  {question}

  ### Retrieved Context:
  1. {retrieved_responses[0]}
  2. {retrieved_responses[1]}
  3. {retrieved_responses[2]}

  ### AI-Generated Answer:
  {llm_response}

  ---

  PART 1: Relevance of Retrieval
  - Assess whether each chunk is relevant to the question.
  - For each chunk, state:
  - Relevance (Yes/No)
  - Reason

  PART 2: Faithfulness of Retrieval
  - Break the generated answer into **distinct factual claims**.
  - For each claim:
  - Claim text
  - Is it supported by retrieved content? (Yes/No)
  - Which chunk(s) support it (if any)

  - Then calculate:
  Faithfulness Score = (Number of Supported Claims) / (Total Claims)

  PART 3: LLM Response
  Please rate the generated answer on a scale of 1 to 5 for each of the following:

  - Correctness: Is it factually accurate compared to the ground truth?
  - Relevance: Does it focus on the core points?
  - Coherence: Is it logically and clearly written?
  - Completeness: Does it match the full scope of the ground truth?
  - Faithfulness: Does the answer stay grounded in the retrieved context and avoid hallucinations?

  ### Format:
  PART 1: Relevance of Chunks
  - Chunk 1: Relevant: <Yes/No> – <reason>
  - Chunk 2: Relevant: <Yes/No> – <reason>
  - Chunk 3: Relevant: <Yes/No> – <reason>

  PART 2: Faithfulness
  - Claim 1: "...", Supported: Yes, Source: Chunk 2
  - Claim 2: "...", Supported: No
  ...
  Faithfulness Score: X/Y = Z.ZZ

  PART 3: LLM Response
  -Correctness: <score>/5 - <comment>
  -Relevance: <score>/5 - <comment>
  -Coherence: <score>/5 - <comment>
  -Completeness: <score>/5 - <comment>
  -Faithfulness: <score>/5 - <comment>

  """

  eval = generator([{"role": "user", "content": eval_prompt}])
  print("\n Retrieval Evaluation:")
  print("="*35)
  print(eval[0]["generated_text"])
  print("="*35)

  return semantic_response, llm_response

In [ ]:
model_names = [
  "Qwen/Qwen2.5-3B-Instruct",
  "microsoft/phi-2",
  "meta-llama/Llama-3.2-3B-Instruct"

]

questions = [
    "What is the Master Theorem?",
    "Explain what is meant by Divide and Conquer algorithms.",
    "What is the Knapsack Problem?"
]

ground_truths = [
       "Master Theorem: If f(n) ∈ Θ(n^d) where d ≥ 0 in recurrence (5.1), then\n"
    "T(n) ∈\n"
    "    Θ(n^d)         if a < b^d,\n"
    "    Θ(n^d log n)   if a = b^d,\n"
    "    Θ(n^log_b a)   if a > b^d.\n"
    "Analogous results hold for the O and Ω notations, too.",

    "Divide-and-conquer is a general algorithm design technique that solves a problem by dividing it into several smaller subproblems of the same type (ideally, of about equal size), solving each of them recursively, and then combining their solutions to get a solution to the original problem. Many efficient algorithms are based on this technique, although it can be both inapplicable and inferior to simpler algorithmic solutions.",

    "The knapsack problem can be posed as follows. Given a knapsack of capacity W and n items of weights w1,...,wn and values v1,...,vn, find the most valuable subset of the items that fits into the knapsack.",
]

In [ ]:
rag_pipeline(model_names[0], questions[0], ground_truths[0] )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Model Information
torch.bfloat16
Total Parameters: 3085.938688 million
Estimated Memory Footprint: 5885.96 MB

============Dense Embeddings=============
page 518 - Score: 0.3644 - Short Tutorial on Recurrence Relations 491
logb n∑
j=1
(bd/a)j = (bd/a)(bd/a)logb n − 1
(bd/a) − 1 i...
page 511 - Score: 0.3639 - 484 Short Tutorial on Recurrence Relations
After some standard algebraic simpliﬁcations, we get the ...
page 517 - Score: 0.3623 - 490 Short Tutorial on Recurrence Relations
where b ≥ 2, then
T (n)∈ /Theta1(f (n)).
(The analogous r...
************BM25 Results*************
page 200 - Score: 12.2036 - Hence, according to the Master Theorem, Cworst (n) ∈ /Theta1(nlog n) (why?). In fact,
it is easy to ...
page 547 - Score: 11.9289 - 520 Hints to Exercises
4. Look at the notations used in the theorem’s statement.
5. Apply the Master...
page 548 - Score: 11.8900 - the text.
9. The recurrence for the number of multiplications in Pan’s algorithm is similar
to that ...
BM25 1.0
BM25 0.5
BM

('Page 200: Hence, according to the Master Theorem, Cworst (n) ∈ /Theta1(nlog n) (why?). In fact,\nit is easy to ﬁnd the exact solution to the worst-case recurrence for n = 2k:\nCworst (n) = n log2 n − n + 1.-------Page 518: Short Tutorial on Recurrence Relations 491\nlogb n∑\nj=1\n(bd/a)j = (bd/a)(bd/a)logb n − 1\n(bd/a) − 1 if bd ̸= a\nand\nlogb n∑\nj=1\n(bd/a)j = logb n if bd = a.\nIf a<b d, then bd/a >1, and therefore\nlogb n∑\nj=1\n(bd/a)j = (bd/a)(bd/a)logb n − 1\n(bd/a) − 1 ∈ /Theta1((bd/a)logb n).\nHence, in this case,\nT (n)= nlogb a[T( 1) +\nlogb n∑\nj=1\n(bd/a)j] ∈ nlogb a/Theta1((bd/a)logb n)\n= /Theta1(nlogb a(bd/a)logb n) = /Theta1(alogb n(bd/a)logb n)\n= /Theta1(bd logb n) = /Theta1(blogb nd\n) = /Theta1(nd).\nIf a>b d, then bd/a <1, and therefore\nlogb n∑\nj=1\n(bd/a)j = (bd/a)(bd/a)logb n − 1\n(bd/a) − 1 ∈ /Theta1(1).\nHence, in this case,\nT (n)= nlogb a[T( 1) +\nlogb n∑\nj=1\n(bd/a)j] ∈ /Theta1(nlogb a).\nIf a = bd, then bd/a = 1, and therefore\nT (n)= nlogb a[T( 1) 

In [ ]:
rag_pipeline(model_names[0], questions[1], ground_truths[1] )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Model Information
torch.bfloat16
Total Parameters: 3085.938688 million
Estimated Memory Footprint: 5885.96 MB

============Dense Embeddings=============
page 196 - Score: 0.2009 - 5
Divide-and-Conquer
Whatever man prays for, he prays for a miracle. Every prayer reduces itself
to ...
page 198 - Score: 0.1940 - Divide-and-Conquer 171
As mentioned above, in the most typical case of divide-and-conquer a prob-
le...
page 197 - Score: 0.1910 - 170 Divide-and-Conquer
subproblem 1
of size n/2
subproblem 2
of size n/2
solution to
subproblem 1
so...
************BM25 Results*************
page 32 - Score: 12.8250 - Step 1 Assign the value of min{m, n} to t.
Step 2 Divide m by t. If the remainder of this division i...
page 406 - Score: 11.8891 - 10.3 Maximum Matching in Bipartite Graphs 379
3. a. What is the largest and what is the smallest pos...
page 364 - Score: 11.0472 - theFibonacci heap (e.g., [Cor09]). However, its complexity and a considerable
overhead make such an ...
BM25 1.0
BM25 0.5
BM2

('Page 32: Step 1 Assign the value of min{m, n} to t.\nStep 2 Divide m by t. If the remainder of this division is 0, go to Step 3;\notherwise, go to Step 4.\nStep 3 Divide n by t. If the remainder of this division is 0, return the value of\nt as the answer and stop; otherwise, proceed to Step 4.\nStep 4 Decrease the value of t by 1. Go to Step 2.\nNote that unlike Euclid’s algorithm, this algorithm, in the form presented,\ndoes not work correctly when one of its input numbers is zero. This example-------Page 196: 5\nDivide-and-Conquer\nWhatever man prays for, he prays for a miracle. Every prayer reduces itself\nto this—Great God, grant that twice two be not four.\n—Ivan Turgenev (1818–1883), Russian novelist and short-story writer\nD\nivide-and-conquer is probably the best-known general algorithm design\ntechnique. Though its fame may have something to do with its catchy name, it\nis well deserved: quite a few very efﬁcient algorithms are speciﬁc implementations\nof this general strate

In [ ]:
rag_pipeline(model_names[0], questions[2], ground_truths[2] )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Model Information
torch.bfloat16
Total Parameters: 3085.938688 million
Estimated Memory Footprint: 5885.96 MB

============Dense Embeddings=============
page 320 - Score: 0.2420 - 8.2 The Knapsack Problem and Memory Functions 293
of solutions to its smaller subinstances. Let us c...
page 463 - Score: 0.2400 - 436 Coping with the Limitations of Algorithm Power
Before we leave the assignment problem, we have t...
page 145 - Score: 0.2027 - 118 Brute Force and Exhaustive Search
item 4item 3item 2item 1knapsack
10
w1 = 7
v1 = $42
w2 = 3
v2 ...
************BM25 Results*************
page 35 - Score: 11.2526 - Euclid’s algorithm compared with the algorithm based on checking con-
secutive integers from min{m, ...
page 481 - Score: 11.0401 - 454 Coping with the Limitations of Algorithm Power
and branch-and-bound (Section 12.2). Now we will ...
page 322 - Score: 10.8225 - 8.2 The Knapsack Problem and Memory Functions 295
or worse). The classic dynamic programming approac...
BM25 1.0
BM25 0.5
BM2

('Page 35: Euclid’s algorithm compared with the algorithm based on checking con-\nsecutive integers from min{m, n} down to gcd(m, n).\n7. Prove the equality gcd (m, n) = gcd (n, mmod n) for every pair of positive\nintegers m and n.\n8. What does Euclid’s algorithm do for a pair of integers in which the ﬁrst is\nsmaller than the second? What is the maximum number of times this can\nhappen during the algorithm’s execution on such an input?\n9. a. What is the minimum number of divisions made by Euclid’s algorithm-------Page 320: 8.2 The Knapsack Problem and Memory Functions 293\nof solutions to its smaller subinstances. Let us consider an instance deﬁned by the\nﬁrst i items, 1 ≤ i ≤ n, with weights w1,...,w i, values v1,...,v i, and knapsack\ncapacity j, 1 ≤ j ≤ W. Let F(i,j) be the value of an optimal solution to this\ninstance, i.e., the value of the most valuable subset of the ﬁrst i items that ﬁt into\nthe knapsack of capacity j. We can divide all the subsets of the ﬁrsti items that 

In [ ]:
rag_pipeline(model_names[1], questions[0], ground_truths[0] )

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Device set to use cuda:0


Model Information
torch.float16
Total Parameters: 2779.68384 million
Estimated Memory Footprint: 5301.83 MB

============Dense Embeddings=============
page 518 - Score: 0.3644 - Short Tutorial on Recurrence Relations 491
logb n∑
j=1
(bd/a)j = (bd/a)(bd/a)logb n − 1
(bd/a) − 1 i...
page 511 - Score: 0.3639 - 484 Short Tutorial on Recurrence Relations
After some standard algebraic simpliﬁcations, we get the ...
page 517 - Score: 0.3623 - 490 Short Tutorial on Recurrence Relations
where b ≥ 2, then
T (n)∈ /Theta1(f (n)).
(The analogous r...
************BM25 Results*************
page 200 - Score: 12.2036 - Hence, according to the Master Theorem, Cworst (n) ∈ /Theta1(nlog n) (why?). In fact,
it is easy to ...
page 547 - Score: 11.9289 - 520 Hints to Exercises
4. Look at the notations used in the theorem’s statement.
5. Apply the Master...
page 548 - Score: 11.8900 - the text.
9. The recurrence for the number of multiplications in Pan’s algorithm is similar
to that ...
BM25 1.0
BM25 0.5
BM25

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
rag_pipeline(model_names[1], questions[1], ground_truths[1] )

In [ ]:
rag_pipeline(model_names[1], questions[2], ground_truths[2] )

In [ ]:
rag_pipeline(model_names[2], questions[0], ground_truths[0] )

In [ ]:
rag_pipeline(model_names[2], questions[1], ground_truths[1] )

In [ ]:
rag_pipeline(model_names[2], questions[2], ground_truths[2] )